In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import spacy
import re
import string
import torchtext
from nltk.tokenize import word_tokenize

import sklearn
from sklearn.model_selection import train_test_split

import numpy as np
import tqdm
import pandas as pd
import time

import os
import gc
import scipy

# Reference: https://www.kaggle.com/code/rahmafarag/ml-translate

## Load Data

In [2]:
# Load cleaned data directly if available
if os.path.isfile('./data/cleaned_data.csv'):
    df = pd.read_csv('./data/cleaned_data.csv', index_col=False)
else:

    start = time.time()
    df = pd.read_csv('./data/en-fr.csv')

    end = time.time()

    display(end - start)

df.head()

,en,fr
0,sos changing lives changing society how it wor...,sos il a transform notre vie il a transform la...
1,sos site map eos,sos plan du site eos
2,sos feedback eos,sos rtroaction eos
3,sos credits eos,sos crdits eos
4,sos franais eos,sos english eos


## Clean Data

In [3]:
### See reference

def to_lowercase(text):
    return text.lower()

# Remove website links
def remove_links(text):
    template = re.compile(r'https?://\S+|www\.\S+') 
    text = template.sub(r'', text)
    return text

# Remove HTML tags
def remove_html(text):
    template = re.compile(r'<[^>]*>') 
    text = template.sub(r'', text)
    return text


# Remove stopwords
def remove_stopwords(words, stop_words):
    return [word for word in words if word not in stop_words]

# Remove none ascii characters
def remove_non_ascii(text):
    template = re.compile(r'[^\x00-\x7E]+') 
    text = template.sub(r'', text)
    return text

# Replace none printable characters
def remove_non_printable(text):
    template = re.compile(r'[\x00-\x1F]+') 
    text = template.sub(r' ', text)
    return text

# Remove special characters
def remove_special_chars(text):
        text = re.sub("'s", '', text)
        template = re.compile('["#$%&\'()\*\+-/:;<=>@\[\]\\\\^_`{|}~]') 
        text = template.sub(r' ', text)
        return text

# Replace multiple punctuation 
def replace_multiplt_punc(text):
        text = re.sub('[.!?]{2,}', '.', text)
        text = re.sub(',+', ',', text)
        return text

    # Remove numbers
def remove_numbers(text):
        text = re.sub('\d+', ' ', text)
        return text

def handle_spaces(text):
    # Remove extra spaces
    text = re.sub('\s+', ' ', text)
    
    # Remove spaces at the beginning and at the end of string
    text = text.strip() 
    
    return text

def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def text2words(text):
      return word_tokenize(text)


def clean_text(text):
    text = remove_special_chars(str(text))
    text = remove_non_ascii(text)
    text = remove_non_printable(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = handle_spaces(text)
    words = text2words(text)
    words = ['sos'] + words + ['eos']

    return ' '.join(words)


9008

,en,fr
450401,sos only with a highly overcompensatory stockr...,sos c est seulement en cas de courbe stockrecr...
450402,sos the model predicts that the assumption mad...,sos le modle prvoit que l hypothse mise au suj...
450403,sos overall the results confirm the unsatisfac...,sos dans l ensemble les rsultats confirment le...
450404,sos error file not found sorry but the file lv...,sos erreur fichier introuvable nous sommes dso...
450405,sos british columbia lodging and campgrounds a...,sos british columbia lodging and campgrounds a...


In [30]:
# clean data only if didn't load cleaned_data
if not os.path.isfile('./data/cleaned_data.csv'):
    train_portion = 0.01
    test_portion = 0.01

    num_train = int(len(df) * train_portion)
    num_test = int(len(df) * test_portion)


    df_subset = pd.concat((df.iloc[:num_train, :], df.iloc[-num_test:, :]))

    display(len(df_subset))

    df_subset.tail()
    start = time.time()

    df_subset['en'] = df_subset['en'].apply(lambda x: clean_text(x))
    df_subset['fr'] = df_subset['fr'].apply(lambda x: clean_text(x))
    end = time.time()

    display(end - start)

    df_subset.to_csv('./data/cleaned_data.csv', index=False)
else:
    df_subset = df.copy()
    num_train = len(df_subset) // 2
    num_test = len(df_subset) // 2

df_subset.head()

,en,fr
0,sos changing lives changing society how it wor...,sos il a transform notre vie il a transform la...
1,sos site map eos,sos plan du site eos
2,sos feedback eos,sos rtroaction eos
3,sos credits eos,sos crdits eos
4,sos franais eos,sos english eos


In [31]:
len(df_subset)

450406

## Construct en and fr vocabulary

In [32]:
english_vocab = []
for sent in df_subset['en']:
    for token in sent.split():
        english_vocab.append(token)
set_english_vocab = set(english_vocab)
len(set_english_vocab)

144176

In [33]:
fr_vocab = []
for sent in df_subset['fr']:
    for token in sent.split():
        fr_vocab.append(token)
set_fr_vocab = set(fr_vocab)
len(set_fr_vocab)

168281

In [34]:
display(english_vocab[:10])
display(fr_vocab[:10])

['sos',
 'changing',
 'lives',
 'changing',
 'society',
 'how',
 'it',
 'works',
 'technology',
 'drives']

['sos', 'il', 'a', 'transform', 'notre', 'vie', 'il', 'a', 'transform', 'la']

In [35]:
input_words = sorted(list(set_english_vocab))
target_words = sorted(list(set_fr_vocab))


num_encoder_tokens = len(set_english_vocab)
num_decoder_tokens = len(set_fr_vocab)
num_decoder_tokens +=1
print(num_encoder_tokens, num_decoder_tokens)

144176 168282


In [36]:
# construct word to index mapping
en_word2index = dict([(word, i+1) for i, word in enumerate(input_words)])
fr_word2index = dict([(word, i+1) for i, word in enumerate(target_words)])

# construct index to word mapping
en_index2word = dict([(i, word) for word, i in en_word2index.items()]) 
fr_index2word = dict([(i, word) for word, i in fr_word2index.items()]) 

In [37]:
en_index2word

{1: 'a',
 2: 'aa',
 3: 'aaa',
 4: 'aaarated',
 5: 'aaas',
 6: 'aabb',
 7: 'aabbhchhch',
 8: 'aabc',
 9: 'aac',
 10: 'aacc',
 11: 'aaccdd',
 12: 'aaccnet',
 13: 'aachen',
 14: 'aacr',
 15: 'aacs',
 16: 'aacti',
 17: 'aactivity',
 18: 'aacute',
 19: 'aad',
 20: 'aadac',
 21: 'aadt',
 22: 'aae',
 23: 'aaf',
 24: 'aafanb',
 25: 'aafc',
 26: 'aafcapril',
 27: 'aafccanada',
 28: 'aafcpromoting',
 29: 'aafcreport',
 30: 'aafcs',
 31: 'aafcthe',
 32: 'aafcwww',
 33: 'aag',
 34: 'aagreement',
 35: 'aah',
 36: 'aahar',
 37: 'aaharindia',
 38: 'aahrp',
 39: 'aahuad',
 40: 'aai',
 41: 'aaib',
 42: 'aairp',
 43: 'aaj',
 44: 'aal',
 45: 'aalborg',
 46: 'aalexx',
 47: 'aalexxintl',
 48: 'aalge',
 49: 'aall',
 50: 'aalogine',
 51: 'aama',
 52: 'aamjiwnaang',
 53: 'aamp',
 54: 'aamt',
 55: 'aanderaa',
 56: 'aandrews',
 57: 'aapausa',
 58: 'aapc',
 59: 'aapcs',
 60: 'aapex',
 61: 'aapq',
 62: 'aar',
 63: 'aarab',
 64: 'aardvark',
 65: 'aarestrup',
 66: 'aargau',
 67: 'aargauer',
 68: 'aarhus',
 69: 'aar

In [38]:
en_len = [len(sent.split()) for sent in df_subset['en']]
fr_len = [len(sent.split()) for sent in df_subset['fr']]

en_len = np.array(en_len)
fr_len = np.array(fr_len)


In [39]:
display(np.percentile(en_len, 80))
display(np.percentile(fr_len, 80))

35.0

40.0

In [40]:
max_en_len = int(np.percentile(en_len, 80))
max_fr_len = int(np.percentile(fr_len, 80))

In [42]:
len(df_subset)

450406

## Split data and construct model input & outputs

In [41]:
train_df = df_subset[:num_train]
test_df = df_subset[-num_test:]

X_train = train_df['en']
y_train = train_df['fr']

X_test = test_df['en']
y_test = test_df['fr']

X_train

0         sos changing lives changing society how it wor...
1                                          sos site map eos
2                                          sos feedback eos
3                                           sos credits eos
4                                           sos franais eos
                                ...                        
225198    sos download winter spring issue pdf kb in thi...
225199                         sos expert roundtable on eos
225200    sos making canada the destination of choice fo...
225201    sos national political infrastructure and fore...
225202                      sos the productivity volume eos
Name: en, Length: 225203, dtype: object

In [92]:
gc.collect()

3705

In [106]:
batch_size=len(X_train)
encoder_input_data = np.zeros((batch_size, max_en_len), dtype='float32')
decoder_input_data = np.zeros((batch_size, max_fr_len), dtype='float32')
#decoder_target_data = np.zeros((batch_size, max_fr_len, num_decoder_tokens), dtype='float16')
#decoder_target_data = np.zeros((batch_size, max_fr_len), dtype='float32')
decoder_target_data = scipy.sparse.csr_array(shape=(batch_size, max_fr_len, num_decoder_tokens), dtype='float32')

for i in range(batch_size):
    en = X_train[i]
    fr = y_train[i]
    
    for t, word in enumerate(en.split()):
        if t < max_en_len:
            encoder_input_data[i, t] = input_token_index[word]
            
    for t, word in enumerate(fr.split()):
        if ((t < (len(fr.split())-1)) and (t< max_fr_len) ): #we don't include the end token in decoder inputs
            decoder_input_data[i, t] = target_token_index[word]
        if ((t > 0 )and (t<max_fr_len)): # we don;t incluse start token in decoder outputs
            decoder_target_data[i, t-1, target_token_index[word]] = 1.
            #decoder_target_data[i, t-1] = target_token_index[word]
            
display(X_train[0])
display(input_token_index['changing'])
display(encoder_input_data[0])

display(y_train[0])
display(target_token_index['notre'])
display(decoder_input_data[0])

gc.collect()


TypeError: _cs_matrix.__init__() missing 1 required positional argument: 'arg1'

In [28]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, data, tok_en, tok_fr, subset = False, portion=0.01):
        """
        data: csv file that has english and french sentences
        subset: flag to signal if to only use a subset of the data to save processing time
        portion: portion of the data to use if subset is true
        toked_data: path to already tokenized data
        """
        
        pass
        
    def __len__(self):
        pass
        
                
    def __getitem__(self, idx):
        """
        return:
            en: tokenized list of english sentence
            fr: tokenized lise of french sentence
        """
        pass

class TextTestDataset(torch.utils.data.Dataset):
    def __init__(self, data, tok_en, tok_fr, subset = False, portion=0.10):
        """
        data: csv file that has english and french sentences
        subset: flag to signal if to only use a subset of the data to save processing time
        portion: portion of the data to use if subset is true
        """
        pass
        
        
    def __len__(self):
        pass
        
                
    def __getitem__(self, idx):
        """
        return:
            en: tokenized list of english sentence
            fr: tokenized lise of french sentence
        """
        pass
        

